---
### Hi! My name is Lucas Pereira, and I'm the creator of this notebook. It's great to have you here!

You can find the original project on my GitHub and connect with me on LinkedIn.

* **Original notebook:** [https://github.com/dsandux/AB-Test-Toolkit](https://github.com/dsandux/AB-Test-Toolkit)
* **LinkedIn:** [https://www.linkedin.com/in/lucaspereira](https://www.linkedin.com/in/lucaspereira)
---

# A/B Test Analysis with Bayesian Statistics
This notebook performs an end-to-end analysis of A/B test results using Bayesian statistical methods. The objective is to go beyond the traditional "p-value" to calculate the probability of each variant being the best and to quantify the expected risk associated with choosing one variant over another.

#### 1. Setup

Here is where we install all the libraries needed to run this test.


In [ ]:
# 1. Setup: Load Libraries
# This cell installs and imports all the necessary Python libraries for the analysis,
# visualization, and interactive controls.
# You only need to run this cell once per session.

# --- Install required packages ---
!pip install -q pandas numpy scipy matplotlib seaborn ipywidgets

# --- Import Libraries ---
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import PercentFormatter, FuncFormatter
import os

# --- New libraries for interactive widgets and file handling ---
import ipywidgets as widgets
from IPython.display import display, clear_output
import io

print("✅ Libraries loaded successfully. You can now proceed to the Control Panel.")


#### 2. Upload files and set parameters

Run the cell below to see the upload file and settings form. The default settings are better for most of cases (unless you have a specific need and knows what the options means). You can just upload your Excel file and run the cell.

In [ ]:
# --- 2. Control Panel ---
# Use the controls below to configure your test. After making your selections,
# click the "Confirm Selections" button to prepare the data for analysis.

import ipywidgets as widgets
from IPython.display import display, clear_output
from datetime import datetime
import os
import io

# --- Define Layouts for a cleaner and more professional look ---
# This helps align widgets and give them consistent sizing and spacing.
wide_layout = widgets.Layout(width='98%', margin='5px 0')
button_layout = widgets.Layout(width='250px', margin='15px 0 0 0')

# --- Create Interactive Widgets ---

# File Uploader Widget
uploader = widgets.FileUpload(
    accept='.xlsx',
    description='Upload Data File',
    button_style='info',
    tooltip='Click to upload your A/B test data in .xlsx format',
    layout=wide_layout
)

# Bayesian Prior Selector Widget
prior_selector = widgets.Dropdown(
    options=[
        ('Jeffreys (Recommended for most tests)', (0.5, 0.5)),
        ('Uniform (Neutral / Uninformative)', (1.0, 1.0))
    ],
    value=(0.5, 0.5),
    description='Bayesian Prior:',
    style={'description_width': 'initial'},
    tooltip='Choose the starting assumption for the model.',
    layout=wide_layout
)

# Risk Threshold Slider Widget
risk_slider = widgets.FloatSlider(
    value=0.01,
    min=0.005,
    max=0.05,
    step=0.005,
    description='Risk Tolerance:',
    style={'description_width': 'initial'},
    readout_format='.2%',
    tooltip='Lower = more cautious. Higher = more aggressive.',
    layout=wide_layout
)

# --- Add Button and Output Area for Feedback ---

# Button to confirm the upload and parameter settings
confirm_button = widgets.Button(
    description="Confirm Selections",
    button_style='success',
    tooltip='Click to confirm your file and settings',
    icon='check',
    layout=button_layout
)

# Output widget to display feedback messages (success or error)
output_area = widgets.Output()

# --- Define the logic for the button click ---
def on_confirm_button_clicked(b):
    with output_area:
        clear_output(wait=True) # Clear previous messages

        # Add a try-except block for robust error handling
        try:
            # Check if a file has been uploaded
            if not uploader.value:
                print("❌ Error: Please upload a data file before confirming.")
                return

            # Access the uploaded file's data and name
            uploaded_file_dict = uploader.value[0]
            original_name = uploaded_file_dict['name']

            # Create the new filename with the current date
            date_str = datetime.now().strftime("%Y-%m-%d")
            base_name, extension = os.path.splitext(original_name)
            new_filename_with_date = f"{base_name}_{date_str}{extension}"

            # Store the data and new filename in global variables
            global uploaded_data_content, confirmed_filename
            uploaded_data_content = io.BytesIO(uploaded_file_dict['content'])
            confirmed_filename = new_filename_with_date

            # Provide success feedback to the user
            print(f"✅ Success! File '{original_name}' is ready for analysis.")
            print(f"   It will be referred to as '{confirmed_filename}' in this session.")
            print("\nYou may now proceed to the next cell.")

        except Exception as e:
            # If any error occurs, print it clearly for debugging.
            print("❌ An unexpected error occurred. Please check the details below:")
            print(f"   Error Type: {type(e).__name__}")
            print(f"   Error Details: {e}")


# Link the function to the button's click event
confirm_button.on_click(on_confirm_button_clicked)

# --- Display the final Control Panel using a VBox and Accordion for a structured layout ---
# We use an Accordion to tuck away the more advanced settings, simplifying the UI.
advanced_settings = widgets.Accordion(
    children=[widgets.VBox([prior_selector, risk_slider])],
    selected_index=None # Start with the accordion closed
)
advanced_settings.set_title(0, 'Advanced Settings (Prior & Risk)')
advanced_settings.layout = wide_layout


# The main VBox organizes all the elements vertically and adds a border.
control_panel_layout = widgets.VBox([
    widgets.HTML("<h2 style='font-family: Arial, sans-serif;'>Step 1: Configure Your Test</h2>"),
    widgets.HTML("<b style='font-family: Arial, sans-serif;'>Upload your data file and adjust the settings below, then click Confirm.</b>"),
    uploader,
    advanced_settings,
    confirm_button,
    output_area
], layout=widgets.Layout(
    border='1px solid #ccc',
    padding='15px',
    border_radius='8px',
    margin='10px 0'
))

display(control_panel_layout)


#### 3. Data Loading and Validation
This cell handles the critical first step of loading and validating the A/B test data. The code will read the specified Excel file, confirm its existence, and verify that it contains the necessary columns for the analysis: variant, reach, and conversion. This ensures the data is correctly structured before proceeding.

In [ ]:
# --- 3. Data Loading and Validation ---
# This cell loads the data you confirmed in the Control Panel into a pandas
# DataFrame. It also validates that the file contains the required columns
# ('variant', 'reach', 'conversion').

# --- Define Expected Data Structure ---
REQUIRED_COLUMNS = ['variant', 'reach', 'conversion']

# --- Validation and Loading Logic ---
# First, check if the 'uploaded_data_content' variable exists.
# This variable is created only when you click "Confirm Selections" in the Control Panel.
if 'uploaded_data_content' not in locals():
    print("❌ Error: No data file has been confirmed yet.")
    print("   Please go back to the Control Panel, upload a file, and click 'Confirm Selections'.")

else:
    # If the data exists, try to load and validate it.
    try:
        print(f"Attempting to load data from '{confirmed_filename}'...")

        # Load the data from the in-memory variable into a pandas DataFrame.
        # We use 'uploaded_data_content' instead of a file path.
        df = pd.read_excel(uploaded_data_content)

        # Check if all the required columns are in the DataFrame.
        if all(col in df.columns for col in REQUIRED_COLUMNS):
            # If validation is successful, print a confirmation and the data's head.
            print("✅ Success: File loaded and validated.")
            print("\n--- First 5 Rows of the Dataset ---")
            print(df.head().to_string(index=False))
        else:
            # If columns are missing, identify them and report the error.
            missing_cols = [col for col in REQUIRED_COLUMNS if col not in df.columns]
            print(f"---")
            print(f"❌ Error: Missing Columns.")
            print(f"The file '{confirmed_filename}' was loaded, but is missing required columns.")
            print(f"   - Missing column(s): {missing_cols}")
            print(f"   - Please ensure the file contains all of the following columns: {REQUIRED_COLUMNS}")

    except Exception as e:
        # Catch any other potential errors during the file reading process.
        print(f"---")
        print(f"❌ An unexpected error occurred while reading the file: {e}")


#### 4. Calculation of Posterior Parameters
Here, we perform the core Bayesian update for our A/B test. 🧪

In [ ]:
# --- 4. Calculation of Posterior Parameters ---
# This cell performs the core Bayesian update. It combines the prior belief
# you selected in the Control Panel with the observed data (reach and conversions)
# to calculate the posterior distribution for each variant.

# First, check if the DataFrame 'df' exists from the previous step.
if 'df' not in locals():
    print("❌ Error: DataFrame 'df' not found.")
    print("   Please run the 'Data Loading and Validation' cell successfully before proceeding.")
else:
    try:
        # Get the selected prior values (alpha, beta) from the Control Panel widget
        PRIOR_ALPHA, PRIOR_BETA = prior_selector.value

        # --- Apply the Beta-Binomial Conjugate Update Rule ---
        # posterior_alpha = prior_alpha + number_of_successes (conversions)
        # posterior_beta = prior_beta + number_of_failures (reach - conversions)
        df['posterior_alpha'] = PRIOR_ALPHA + df['conversion']
        df['posterior_beta'] = PRIOR_BETA + (df['reach'] - df['conversion'])

        # --- Display the Updated DataFrame ---
        # Show the DataFrame with the newly calculated posterior parameters.
        print("✅ Success: Posterior parameters calculated.")
        print("\n--- DataFrame with Updated Posterior Parameters ---")
        display_cols = ['variant', 'reach', 'conversion', 'posterior_alpha', 'posterior_beta']
        print(df[display_cols].to_string(index=False))

    except Exception as e:
        print(f"❌ An unexpected error occurred: {e}")



#### 5. Generation of the Posterior Plot

This cell generates the most important visualization for our analysis: the posterior probability distributions. The ridgeline plot is used for better readability when comparing multiple variants.

In [ ]:
# --- 5. Generation of the Posterior Plot ---
# This cell generates the most important visualization for our analysis: the
# posterior probability distributions. The ridgeline plot is used for better
# readability when comparing multiple variants.

# First, check if the DataFrame 'df' with posterior parameters exists.
if 'df' in locals() and 'posterior_alpha' in df.columns:
    try:
        # --- 1. Setup for Ridgeline Plot ---
        # We sort by the posterior mean to have a more organized plot.
        # The posterior mean is the average of the distribution.
        if 'posterior_mean' not in df.columns:
             df['posterior_mean'] = df['posterior_alpha'] / (df['posterior_alpha'] + df['posterior_beta'])

        sorted_df = df.sort_values('posterior_mean', ascending=False)

        # Create a figure and axes for the plot.
        fig, ax = plt.subplots(figsize=(12, 2 + len(sorted_df) * 0.7))

        # --- 1a. Create a color palette ---
        # Use a colormap to get a unique color for each variant.
        colors = plt.cm.viridis(np.linspace(0.1, 0.9, len(sorted_df)))

        # --- 2. Dynamically Determine X-axis Range ---
        # Ensure all distributions and their credible intervals are fully visible.
        max_x = 0
        for i, row in sorted_df.iterrows():
            percentile_999 = stats.beta.ppf(0.999, row['posterior_alpha'], row['posterior_beta'])
            if percentile_999 > max_x:
                max_x = percentile_999
        x = np.linspace(0, max_x * 1.05, 1000)

        # --- 3. Plot Each Variant as a Ridge ---
        y_offset_step = 0.8  # Controls vertical spacing between ridges

        for i, (row, color) in enumerate(zip(sorted_df.itertuples(), colors)):
            y_offset = i * y_offset_step

            # Calculate the Probability Density Function (PDF)
            pdf = stats.beta.pdf(x, row.posterior_alpha, row.posterior_beta)

            # Plot the main distribution curve with a label for the legend.
            ax.plot(x, pdf + y_offset, color=color, lw=1.5, label=row.variant)

            # Add a light fill for the entire distribution
            ax.fill_between(x, y_offset, pdf + y_offset, alpha=0.2, color=color)

            # --- 4. Calculate and Shade the 95% Credible Interval ---
            # This interval contains the true conversion rate with 95% probability.
            ci_low, ci_high = stats.beta.ppf([0.025, 0.975], row.posterior_alpha, row.posterior_beta)

            # Create a mask for the x-values within the credible interval
            ci_mask = (x >= ci_low) & (x <= ci_high)

            # Add a darker shade on top for the 95% credible interval
            ax.fill_between(x[ci_mask], y_offset, pdf[ci_mask] + y_offset, alpha=0.4, color=color)

        # --- 5. Finalize and Display the Plot ---
        from matplotlib.patches import Patch

        # Clean up the plot aesthetics
        ax.set_title('Posterior Distributions of Conversion Rates', fontsize=16)
        ax.set_xlabel('Conversion Rate', fontsize=12)
        ax.set_yticks([]) # Hide y-axis ticks as they are not meaningful here
        ax.spines['left'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)

        # Create and display a custom legend
        handles, labels = ax.get_legend_handles_labels()

        # Add "Variant" prefix to each label
        new_labels = [f'Variant {label}' for label in labels]

        # Create a proxy artist for the shaded area to include in the legend
        ci_patch = Patch(color='gray', alpha=0.4, label='95% Credible Interval')
        handles.append(ci_patch)
        new_labels.append('95% Credible Interval')

        # Position the legend above the plot in the top-right corner with a smaller font
        fig.legend(handles, new_labels, title="Legend", bbox_to_anchor=(0.98, 0.98), loc='upper right', fontsize='small')

        plt.tight_layout(rect=[0, 0, 1, 0.95]) # Adjust layout to make space for the title and legend
        plt.show()

    except Exception as e:
        print(f"❌ An unexpected error occurred while generating the plot: {e}")
else:
    print("❌ Error: DataFrame 'df' with posterior parameters not found.")
    print("   Please run the previous cells successfully before proceeding.")


#### 6. # Monte Carlo Simulation

Run this cell to start the Monte Carlo Simulation and create 100.000 random samples.

In [ ]:
# Monte Carlo Simulation

# This cell performs a Monte Carlo simulation to draw random samples from the
# posterior distribution of each variant. These samples will allow us to
# empirically compare the variants and calculate key metrics, such as the
# probability of one being better than the other and the expected loss.

# --- 1. Simulation Setup ---
# Define the number of random samples to generate for each variant's distribution.
# A larger number of samples leads to more stable and accurate estimates of our metrics.
N_SAMPLES = 100000

# We will store the generated samples in a dictionary, with variant names as keys.
posterior_samples = {}


# --- 2. Run Simulation ---
# Check if the dataframe 'df' with posterior parameters exists to avoid errors.
if 'df' in locals() and 'posterior_alpha' in df.columns:

    # Iterate over each variant (row) in the DataFrame.
    for i, row in df.iterrows():
        variant_name = row['variant']
        p_alpha = row['posterior_alpha']
        p_beta = row['posterior_beta']

        # Generate N_SAMPLES from the Beta distribution defined by the variant's
        # posterior parameters. Each sample represents a plausible "true"
        # conversion rate for that variant, according to our model.
        samples = stats.beta.rvs(a=p_alpha, b=p_beta, size=N_SAMPLES)

        # Store the resulting array of samples in our dictionary.
        posterior_samples[variant_name] = samples

    # --- 3. Output: Simulation Summary ---
    # The simulation is complete. The following is a summary of the process.
    print("--- Monte Carlo Simulation Summary ---")
    print(f"✅ Simulation completed successfully.")
    print(f"   - Samples generated per variant: {N_SAMPLES:,}")
    print(f"   - Variants simulated: {list(posterior_samples.keys())}")

    print("\nData Preview (first 3 samples for each variant):")
    for variant, samples in posterior_samples.items():
        preview = [round(s, 6) for s in samples[:3]]
        print(f"  - {variant}: {preview}")

    print("\nThe 'posterior_samples' dictionary is now ready for metric calculation in the next cell.")

else:
    # This message will only be displayed if the prerequisite DataFrame is not found.
    print("Error: DataFrame 'df' with posterior parameters not found.")
    print("Please ensure the data loading (Cell 5) and posterior calculation (Cell 7) were executed successfully.")

#### 6. Calculation and Presentation of Metrics
This is the final calculation step where we translate our simulation results into actionable business metrics. 🏆

In [ ]:
# --- 7. Calculation and Presentation of Metrics ---
# This cell translates our simulation results into actionable business metrics:
# 1. Probability of Being Best: The likelihood that a variant is the true winner.
# 2. Expected Loss (Risk): The cost of being wrong if you choose that variant.

# First, check if the simulation data from the previous step exists.
if 'posterior_samples' in locals():
    try:
        # --- Get the Risk Threshold from the Control Panel ---
        # This line was missing. It retrieves the value set by the designer.
        RISK_THRESHOLD = risk_slider.value

        # --- 1. Combine Simulation Samples into a DataFrame ---
        samples_df = pd.DataFrame(posterior_samples)
        samples_df['max_conversion_rate'] = samples_df.max(axis=1)
        variant_names = list(posterior_samples.keys())

        # --- 2. Calculate 'Probability to be Best' and 'Expected Loss' ---
        results = []
        for variant in variant_names:
            prob_best = (samples_df[variant] == samples_df['max_conversion_rate']).mean()
            loss = samples_df['max_conversion_rate'] - samples_df[variant]
            expected_loss = loss.mean()
            results.append({
                "Variant": variant,
                "Probability to be Best": prob_best,
                "Expected Loss (Risk)": expected_loss
            })

        # --- 3. Format and Display Results as a Table ---
        results_df = pd.DataFrame(results)

        # Create the 'Decision Guide' column based on the threshold
        conditions = [
            results_df['Expected Loss (Risk)'] > RISK_THRESHOLD,
            results_df['Expected Loss (Risk)'] < RISK_THRESHOLD
        ]
        choices = ['Above Threshold', 'Below Threshold']
        results_df['Decision Guide'] = np.select(conditions, choices, default='Equals Threshold')

        # Add other necessary columns from the main 'df' for the final report
        if 'df' in locals():
            results_df = pd.merge(results_df, df[['variant', 'posterior_mean']], left_on='Variant', right_on='variant', how='left').drop('variant', axis=1)

        # Sort the DataFrame by the lowest risk
        results_df = results_df.sort_values(by="Expected Loss (Risk)")

        # --- 4. Final Styling ---
        styled_df = results_df.style.format({
            "Probability to be Best": "{:.2%}",
            "Expected Loss (Risk)": "{:.4%}"
        }).set_properties(**{'text-align': 'center'}) \
        .set_caption(f"🏆 Bayesian A/B Test Results (Risk Threshold: {RISK_THRESHOLD:.1%})") \
        .hide(axis="index")

        # Display the final, styled table
        display(styled_df)
        print("\n(A variant with risk 'Below Threshold' is generally considered a safe choice.)")

    except Exception as e:
        print(f"❌ An unexpected error occurred: {e}")
else:
    print("❌ Error: The 'posterior_samples' dictionary was not found.")
    print("   Please ensure the Monte Carlo Simulation cell was executed successfully.")


#### 7. Automated Conclusion Logic
This final, automated cell translates our statistical results into a clear business recommendation with key metrics. 🎯

In [ ]:
# --- 9. Automated Report & Recommendation ---
# This final cell interprets the results and generates a clear, text-based
# recommendation with detailed explanations of the key metrics, formatted in Markdown.

from IPython.display import display, Markdown
import pandas as pd

# First, check if the results_df DataFrame from the previous step exists.
if 'results_df' not in locals():
    print("❌ Error: The 'results_df' DataFrame was not found.")
    print("   Please run the 'Calculation and Presentation of Metrics' cell successfully before proceeding.")
else:
    try:
        # --- Get Risk Threshold from Control Panel ---
        RISK_THRESHOLD = risk_slider.value

        # --- Ensure all necessary data is in results_df for the report ---
        # This makes the cell robust by checking for and merging only the required columns
        # that are actually missing from the original 'df' DataFrame.
        if 'df' in locals():
            required_cols = ['posterior_mean', 'reach', 'conversion']
            missing_cols = [col for col in required_cols if col not in results_df.columns]

            if missing_cols:
                cols_to_merge = ['variant'] + missing_cols
                # Ensure the original df has the columns before merging
                if all(col in df.columns for col in cols_to_merge):
                    results_df = pd.merge(results_df, df[cols_to_merge], left_on='Variant', right_on='variant', how='left')
                    if 'variant' in results_df.columns:
                         results_df = results_df.drop('variant', axis=1)

        # --- Extract Key Information ---
        best_candidate = results_df.iloc[0]

        is_winner = best_candidate['Expected Loss (Risk)'] < RISK_THRESHOLD

        # --- Build the Markdown Report String in the new order ---
        markdown_report = ""

        # 1. Add the Verdict (Order: 1st)
        if is_winner:
            markdown_report += f"## ✅ Verdict: Test Concluded. Deploy Variant '{best_candidate['Variant']}'.\n"
        else:
            markdown_report += f"## ⚠️ Verdict: Test Inconclusive. Collect More Data.\n"

        markdown_report += "---\n"

        # 2. Add the Stakeholder Summary (Order: 2nd)
        markdown_report += "### Summary for Stakeholders\n"

        if is_winner:
            summary_text = f"The analysis confidently recommends deploying **Variant '{best_candidate['Variant']}'**. It has the highest chance of being the best option, and the risk of choosing it is well below our safety limit of {RISK_THRESHOLD:.1%}. The table in the final section shows the expected performance increase against all other variants."
        else:
            summary_text = f"The results are not yet clear enough to make a confident decision. Our best option still has a risk level higher than our limit. We recommend collecting more data to get a clearer winner."

        markdown_report += summary_text + "\n\n---\n"

        # 3. Add Key Metrics Explained (Order: 3rd)
        markdown_report += "### Key Metrics Explained\n"

        prob_best_str = f"{best_candidate['Probability to be Best']:.1%}"
        markdown_report += f"**🔹 Probability to be Best: {prob_best_str}**\n"
        markdown_report += f"   - *What it means:* This is the probability that Variant '{best_candidate['Variant']}' is truly the best option among all variants tested. A higher percentage means more confidence in it being the winner.\n\n"

        risk_str = f"{best_candidate['Expected Loss (Risk)']:.4%}"
        markdown_report += f"**🔹 Risk (Expected Loss): {risk_str}**\n"
        markdown_report += f"   - *What it means:* This is the 'cost of being wrong.' It represents the average potential drop in conversion rate you would risk by choosing this variant if another one was secretly better. A lower risk is better.\n\n"

        markdown_report += f"**🔹 Expected Uplift**\n"
        markdown_report += f"   - *What it means:* This is the expected percentage increase in the conversion rate of the winning variant compared to another. The table below shows this uplift and the **potential gain in conversions** for the same number of visitors.\n"

        # 4. Add the Expected Uplift Table (Order: 4th, only if there's a winner)
        required_cols_for_table = ['posterior_mean', 'reach', 'conversion']
        if is_winner and len(results_df) > 1 and all(col in results_df.columns for col in required_cols_for_table):
            markdown_report += "\n---\n"
            markdown_report += "### Expected Uplift vs. Other Variants\n"
            markdown_report += "| Compared To | Expected Uplift | Potential Conversion Gain | Total Expected Conversions |\n"
            markdown_report += "|:---|:---|:---|:---|\n"

            # Add the winner's own data as the first row for reference
            winner_conversions = best_candidate['conversion']
            markdown_report += f"| **Variant '{best_candidate['Variant']}' (Winner)** | **-** | **{int(winner_conversions)} conversions (actual)** | **{int(winner_conversions)}** |\n"

            # Loop through all other variants to create the comparison rows
            for i, other_variant in results_df.iloc[1:].iterrows():
                uplift = (best_candidate['posterior_mean'] - other_variant['posterior_mean']) / other_variant['posterior_mean']
                # Corrected Calculation: This shows the *additional* conversions the winner
                # is expected to get compared to the other variant, given the same number of visitors.
                conversion_gain = (best_candidate['posterior_mean'] - other_variant['posterior_mean']) * best_candidate['reach']
                total_expected = winner_conversions + conversion_gain
                markdown_report += f"| Variant '{other_variant['Variant']}' | +{uplift:.2%} | **+{int(round(conversion_gain, 0))}** more conversions | {int(round(total_expected, 0))} |\n"
            markdown_report += "\n"

        # --- Display the final Markdown report ---
        display(Markdown(markdown_report))

    except Exception as e:
        print(f"❌ An unexpected error occurred while generating the report: {e}")
